### Playing with `claude-v1.3-100k`

In [3]:
import os
import anthropic
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import openai
import time


In [4]:
client = anthropic.Client(api_key=os.environ['ANTHROPIC_API_KEY'])

In [24]:
def query_claude(prommpt):
    response = client.completion_stream(
        prompt=f"{anthropic.HUMAN_PROMPT} %s {anthropic.AI_PROMPT}"
        % prommpt,
        stop_sequences=[anthropic.HUMAN_PROMPT],
        max_tokens_to_sample=6000,
        model="claude-v1.3-100k",
        stream=False,
    )
    return response

In [7]:
# load GPTs are GPTs paper

with open('./gpts_are_gpts/main.tex', 'r') as f:
    paper_1 = f.read()

In [29]:
gen_r = query_claude("summerize the paper below: \n\n" + paper_1)

In [30]:
responses = []

for data in gen_r:
    response = data
    responses.append(response)

In [31]:
print(responses[0]['completion'])

 Here is a summary of the paper:

- The paper investigates the potential impact of large language models (LLMs) like GPT on the U.S. labor market. The authors propose a new rubric to assess LLM capabilities and their potential effects on jobs. 

- Using the rubric, the authors find that around 80% of U.S. workers could have at least 10% of their work tasks affected by LLMs, while 19% may see at least 50% of their tasks impacted. The impacts span all wage levels, with higher-income jobs potentially facing greater exposure. 

- The authors compare their rubric to previous studies on technology exposure. They find their measures are correlated with most other studies but explain 28-40% of the unexplained variance, indicating their rubric provides new insights.

- The authors examine exposure by industry and find that information processing industries show high exposure, while manufacturing, agriculture, and mining demonstrate lower exposure. Recent productivity growth appears uncorrelated